In [7]:
import os, json, random, requests, shutil, csv
from sklearn.model_selection import train_test_split
from google.colab import drive

# Monter Google Drive



In [10]:
drive.mount('/content/drive')
base_dir = "/content/drive/MyDrive/Colab Notebooks/PFA"
json_path = f"{base_dir}/incident_subset_1000.json"
output_dir = f"{base_dir}/data"
os.makedirs(output_dir, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
with open(json_path, 'r') as f:
    data = json.load(f)

print(f"✔️ {len(data)} images chargées")

✔️ 1000 images chargées


# Créer les dossiers de catégories


In [12]:
target_categories = {"on fire", "earthquake", "heavy rainfall", "fog"}
for cat in target_categories:
    os.makedirs(os.path.join(output_dir, cat.replace(" ", "_")), exist_ok=True)

#Télécharger et copier les images localement


In [13]:
csv_entries = []
for img in data:
    cat = img["category"]
    if cat not in target_categories:
        continue
    filename = os.path.basename(img["filename"])
    folder = os.path.join(output_dir, cat.replace(" ", "_"))
    save_path = os.path.join(folder, filename)
    try:
        if not os.path.exists(save_path):
            r = requests.get(img["url"], stream=True, timeout=10)
            if r.status_code == 200:
                with open(save_path, 'wb') as f:
                    shutil.copyfileobj(r.raw, f)
        csv_entries.append([save_path, cat])
    except Exception as e:
        print(f"⚠️ Erreur pour {img['url']}: {e}")

print(f"✔️ {len(csv_entries)} images prêtes")

⚠️ Erreur pour https://cdn.winknews.com/wp-content/uploads/2019/11/safety-changes.png: HTTPSConnectionPool(host='cdn.winknews.com', port=443): Max retries exceeded with url: /wp-content/uploads/2019/11/safety-changes.png (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7850c2898410>: Failed to resolve 'cdn.winknews.com' ([Errno -2] Name or service not known)"))
⚠️ Erreur pour http://www.theindependentbd.com/assets/news_images/Typhoon-Damrey.jpg: HTTPSConnectionPool(host='www.theindependentbd.com', port=443): Read timed out. (read timeout=10)
⚠️ Erreur pour https://www.nps.gov/common/uploads/pwr/park/lavo/5658A43F-996F-9951-93CD4D5EF6DACB07/5658A43F-996F-9951-93CD4D5EF6DACB07-large.jpg: HTTPSConnectionPool(host='www.nps.gov', port=443): Read timed out. (read timeout=10)
⚠️ Erreur pour https://cdn-5da890f4f911c8130c44f10c.closte.com/wp-content/uploads/2014/09/Poas-Volcano-in-Clouds-1024x673.jpg: HTTPSConnectionPool(host='cdn-5da890f4f911c8130c44f10c.closte.

#Split train / val / test

In [14]:
filepaths = [x[0] for x in csv_entries]
labels = [x[1] for x in csv_entries]

train_files, temp_files, train_labels, temp_labels = train_test_split(
    filepaths, labels, test_size=0.3, random_state=42, stratify=labels
)
val_files, test_files, val_labels, test_labels = train_test_split(
    temp_files, temp_labels, test_size=1/3, random_state=42, stratify=temp_labels
)

#Sauvegarde CSV


In [15]:
def save_csv(name, paths, labels):
    with open(f"{base_dir}/{name}.csv", 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["filepath", "label"])
        writer.writerows(zip(paths, labels))

save_csv("train", train_files, train_labels)
save_csv("val",   val_files,   val_labels)
save_csv("test",  test_files,  test_labels)

print("✅ CSV générés : train.csv, val.csv, test.csv")

✅ CSV générés : train.csv, val.csv, test.csv
